In [1]:
import os
from langchain.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import Pinecone  # ✅ Correct import for Pinecone
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv()

# ✅ Hugging Face API Key (Ensure it's set in your environment variables)
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not HUGGINGFACE_API_KEY:
    raise ValueError("❌ Missing Hugging Face API Key. Set it as HUGGINGFACEHUB_API_TOKEN")

# ✅ Load DeepSeek-R1 via API Inference
llm = HuggingFaceHub(
    repo_id="deepseek-ai/DeepSeek-R1",  # Model name
    model_kwargs={"temperature": 0.5, "max_length": 2048},
    huggingfacehub_api_token=HUGGINGFACE_API_KEY
)

# ✅ Initialize Pinecone Client
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# ✅ Ensure the new Pinecone index name is set
index_name = os.getenv("PINECONE_INDEX_V2")  # ✅ Updated to use the latest index
if not index_name:
    raise ValueError("❌ PINECONE_INDEX_V2 is not set. Check your .env file.")

# ✅ Ensure the index exists before using it
existing_indexes = [idx["name"] for idx in pinecone_client.list_indexes()]
if index_name not in existing_indexes:
    raise ValueError(f"❌ Index '{index_name}' does not exist in Pinecone. Please create it first.")

# ✅ Initialize Pinecone Index
index = pinecone_client.Index(index_name)

# ✅ Initialize the same embedding model used for storing embeddings
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# ✅ Load the vector store for retrieval
vectorstore = PineconeVectorStore(
    index=index,  # ✅ Connects to the updated Pinecone index
    embedding=huggingface_embeddings,
    text_key="text"  # ✅ Specifies where text metadata is stored
)

# ✅ Define the specific call ID for testing
test_call_id = "call_002"  # Change this to test different calls

# ✅ Create a retriever for call_002 + general PDF solutions
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
        "filter": {
            "$or": [
                {"call_id": test_call_id},
                {"source": "pdf"}
            ]
        }
    }
)



# ✅ Define the Prompt Template for RAG-based Q&A
prompt_template = """
You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. Provide a concise and precise answer.

Provided Information
-------
<ctx>
Context: {context}
</ctx>
-------
<hs>
Chat History: {chat_history}
</hs>
-------
Question: {question}

Useful Answer:
"""


PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "chat_history", "question"]  # ✅ Includes chat history now
)


# ✅ Memory buffer to handle multiple interactions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question"
)

# ✅ Define Retrieval-Based Q&A Chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT, "memory": memory}
)

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\Claudio\AppData\Local\Temp\ipykernel_20380\3906497703.py:20: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
C:\Users\Claudio\AppData\Local\Temp\ipykernel_20380\3906497703.py:43: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingf

In [ ]:
memory.clear() # para resetear memoria (si no se hace, se puede superar el limite de tokens y da error)

In [2]:
query = "how much is the amount per month the client would need to pay?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. Provide a concise and precise answer.

Provided Information
-------
<ctx>
Context: Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That sounds better. What would the payments look like?
Agent: It would be about $30 per month for 12 months.
Client: Okay, but what happens if I miss a payment?

Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t make a 

In [3]:
query = "what are the basic commands of asterisk?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. Provide a concise and precise answer.

Provided Information
-------
<ctx>
Context: de Asterisk.  
VOICEM
AILBAC
KUP  
NA  
Acti
vad
o 
por 
Def
ecto  
NA  
vicidia
l_app.
tgz  
TAR 
Comp
rimid
o  
Backup de los archivos de la 
aplicación de Call Center 
Vicidial.  
VICIDIA
LBACKU
P  
Activ
ado 
por 
Defe
cto  
Acti
vad
o 
por 
Def
ecto  
Acti
vad
o 
por 
Def
ecto  
vicidia
l_soun
dstore
.tgz  
TAR 
Comp
rimid
o

asterisk-rvvvv | grep -e 
6999  
comando para ver la traza solo de una extension  
 Comandos basicos para identificar problemas con 
las ex

In [ ]:
query = "you don't gave me all the basic commands, why? the ones you gave me are more important?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

In [ ]:
query = 'from the solutions guide, give me "Comandos basicos para verificar el estado de las conferencias"'
response = retrievalQA.invoke({"query": query})

print(response["result"])

In [ ]:
query = "what does the RED alarm mean?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

In [ ]:
query = "what is the agent very sorry to hear?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

In [ ]:
query = "how can i fix a queue-stats issue?"
response = retrievalQA.invoke({"query": query})

print(response["result"])